<a href="https://colab.research.google.com/github/xcccl1/BDAO_IMA/blob/main/NLP_pre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Authenticate and access Google Cloud services
from google.colab import auth
auth.authenticate_user()

In [ ]:
# Import the necessary libraries
from google.cloud import storage
from google.cloud import bigquery

# First make connection to BigQuery
# Replace 'your-project-id' with your Google Cloud project ID
# Replace 'your-dataset-name' with your dataset name
project_id = "bdaoima-422710"
dataset_name = "restaurant_data"
table_name = "restaurant_tips" # if you make changes previously, then here you need to make according change

bigquery_client = bigquery.Client(project=project_id)

In [ ]:
# Query all data from the table
query = f"""
    SELECT *
    FROM `{project_id}.{dataset_name}.{table_name}`

"""

query_job = bigquery_client.query(query)
results = query_job.result()

df = query_job.to_dataframe()
df.head()

,user_id,business_id,text,date,compliment_count
0,NBN4MgHP9D3cw--SnauTkA,QoezRbYQncpRqyrLH6Iqjg,They have lots of good deserts and tasty cuban...,2013-02-05 18:35:10+00:00,0
1,DWryovIM4VF2vim4p6WYaw,QoezRbYQncpRqyrLH6Iqjg,Turkey and roast beef,2012-10-11 13:04:02+00:00,0
2,DWryovIM4VF2vim4p6WYaw,QoezRbYQncpRqyrLH6Iqjg,Coffee and baby bread,2012-10-24 12:37:05+00:00,0
3,boRCdL8ez95KwzWe69VoUg,QoezRbYQncpRqyrLH6Iqjg,Such a great place to come and get a Cuban san...,2016-10-02 16:53:40+00:00,0
4,DWryovIM4VF2vim4p6WYaw,QoezRbYQncpRqyrLH6Iqjg,My last time. Continental and a Cuban,2012-11-09 18:51:34+00:00,0


## **Text Cleaning**

In [ ]:
import pandas as pd
# Change your target_column to the column with review data

target_column = "text"

In [ ]:
# drop data with missing values in the 'content' column (target_column)
df = df.dropna(subset=[target_column])

In [ ]:
# convert the relevant column to lowercase
df[target_column] = df[target_column].str.lower()
df

,user_id,business_id,text,date,compliment_count
0,NBN4MgHP9D3cw--SnauTkA,QoezRbYQncpRqyrLH6Iqjg,they have lots of good deserts and tasty cuban...,2013-02-05 18:35:10+00:00,0
1,DWryovIM4VF2vim4p6WYaw,QoezRbYQncpRqyrLH6Iqjg,turkey and roast beef,2012-10-11 13:04:02+00:00,0
2,DWryovIM4VF2vim4p6WYaw,QoezRbYQncpRqyrLH6Iqjg,coffee and baby bread,2012-10-24 12:37:05+00:00,0
3,boRCdL8ez95KwzWe69VoUg,QoezRbYQncpRqyrLH6Iqjg,such a great place to come and get a cuban san...,2016-10-02 16:53:40+00:00,0
4,DWryovIM4VF2vim4p6WYaw,QoezRbYQncpRqyrLH6Iqjg,my last time. continental and a cuban,2012-11-09 18:51:34+00:00,0
...,...,...,...,...,...
649176,tsMF0FcFcHZ8i28WzWtQXw,2VuOLrhe2Iu_FRz1gEIqQA,they are closed at 530 when the sign says 6:00.,2019-07-19 21:30:00+00:00,4
649177,Oi1qbcz2m2SnwUeztGYcnQ,Ht66H1iGx0llc7ne81tjBw,look at the stars people! i'd rather have a ro...,2013-11-18 00:47:06+00:00,4
649178,A4bsa7ykYRVCnb4h2vZALw,3Wy21heeDm8h2tSZfcj6OA,30 minute wait for our drink order is unaccept...,2017-01-15 22:16:30+00:00,5
649179,tsMF0FcFcHZ8i28WzWtQXw,dsfRniRgfbDjC8os848B6A,experience bern's by sitting at the bar too . ...,2020-03-05 01:28:45+00:00,6


In [ ]:
# remove contraction
!pip install contractions
import contractions
df[target_column] = df[target_column].map(lambda x: contractions.fix(x))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 11.2 MB/s eta 0:00:00


In [ ]:
import re


# Remove new line characters ("/n")

df[target_column] = df[target_column].map(lambda x: re.sub('\s+', ' ', x))

# Remove url link
df[target_column] = df[target_column].apply(lambda x: re.sub('http://\S+|https://\S+', '', x))

# Remove email address
df[target_column] = df[target_column].map(lambda x: re.sub('\S*@\S*\s?', '', x))

# Remove non-word characters, so numbers and ___ etc
df[target_column] = df[target_column].str.replace("[^A-Za-z]", " ", regex = True)

# Remove overspace
df[target_column] = df[target_column].map(lambda x: re.sub('\s{2,}', " ", x))

# Remove punctuation
import string
df[target_column] = df[target_column].map(lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x))

## **Text pre-processing**

In [ ]:
import nltk
nltk.download('punkt')
# create a list of the reviews from the 'Review' column
words = df[target_column].tolist()

# tokenise the words
from nltk.tokenize import word_tokenize

word_tokens = [] # create an empty list to put the tokenised words
for review in words:
    word_tokens.append(word_tokenize(review))

word_tokens[0]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


['they',
 'have',
 'lots',
 'of',
 'good',
 'deserts',
 'and',
 'tasty',
 'cuban',
 'sandwiches']

In [ ]:
# Start dealing with stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
# use English stopwords
stopwords = stopwords.words("english")
# you can also extend the stopwords list with more words that you want to remove
stopwords.extend([''])

# create a new list with stop words removed
tokens_without_stopwords = []
for review in word_tokens:
    tokens_without_stopwords.append([w for w in review if not w in stopwords])

tokens_without_stopwords[0]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


['lots', 'good', 'deserts', 'tasty', 'cuban', 'sandwiches']

In [ ]:
# option 2: lemmatise the tokens

nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# The lemmatizer requires the correct POS tag to be accurate, if you use the default settings of the WordNetLemmatizer.lemmatize(), the default tag is noun.
# Which means the words other than noun would not be lemmatised, and this is not what we expect.
# Therefore we need to define the POS tags ('n' for nouns(default), 'a' for adj, 'v' for verb, 'r' for adverb) which are allowed to be lemmatised.

lemmatised = []
for review in tokens_without_stopwords:
  n_lemmatised = [lemmatizer.lemmatize(w) for w in review]
  v_lemmatised = [lemmatizer.lemmatize(w,'v') for w in n_lemmatised]
  r_lemmatised = [lemmatizer.lemmatize(w,'r') for w in v_lemmatised]
  a_lemmatised = [lemmatizer.lemmatize(w,'a') for w in r_lemmatised]
  lemmatised.append(a_lemmatised) # here noun, adj, verb, adv are the POS tags that we allowed

lemmatised[0]



[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


['lot', 'good', 'desert', 'tasty', 'cuban', 'sandwich']

In [ ]:
# put the tokens back together as text after stemming
import string
rejoin = []
for review in lemmatised:
    x = " ".join(review) # join the text back together
    # remove punctuation from the reviews using the string package
    rejoin.append(x)

# add the reformed text to the data frame
df['filtered_review'] = rejoin
# put the tokens back together as text after lemmatisation
import string
rejoin_2 = []
for review in lemmatised: # Here we choose to use stemming instead of lemmatisation
    x = " ".join(review) # join the text back together
    # remove punctuation from the reviews using the string package
    rejoin_2.append(x)

# add the reformed text to the data frame
df['filtered_review_2'] = rejoin_2

In [ ]:
df.head()

,user_id,business_id,text,date,compliment_count,filtered_review,filtered_review_2
0,NBN4MgHP9D3cw--SnauTkA,QoezRbYQncpRqyrLH6Iqjg,they have lots of good deserts and tasty cuban...,2013-02-05 18:35:10+00:00,0,lot good desert tasty cuban sandwich,lot good desert tasty cuban sandwich
1,DWryovIM4VF2vim4p6WYaw,QoezRbYQncpRqyrLH6Iqjg,turkey and roast beef,2012-10-11 13:04:02+00:00,0,turkey roast beef,turkey roast beef
2,DWryovIM4VF2vim4p6WYaw,QoezRbYQncpRqyrLH6Iqjg,coffee and baby bread,2012-10-24 12:37:05+00:00,0,coffee baby bread,coffee baby bread
3,boRCdL8ez95KwzWe69VoUg,QoezRbYQncpRqyrLH6Iqjg,such a great place to come and get a cuban san...,2016-10-02 16:53:40+00:00,0,great place come get cuban sandwich smell brea...,great place come get cuban sandwich smell brea...
4,DWryovIM4VF2vim4p6WYaw,QoezRbYQncpRqyrLH6Iqjg,my last time continental and a cuban,2012-11-09 18:51:34+00:00,0,last time continental cuban,last time continental cuban


In [ ]:
# First, ensure pandas-gbq is installed
!pip install pandas-gbq

# Specify the destination table and dataset in BigQuery
# Replace 'your_dataset_name' with your dataset name where you are sending the new table to
# Replace 'new_table_name' with a name you would like to give to your table
# Replace 'your-project-id' with your project id
destination_table = 'restaurant_data.tips_nlp'
project_id = 'bdaoima-422710'

# Use the to_gbq function to upload the DataFrame (after data processing) to BigQuery
df.to_gbq(destination_table, project_id=project_id, if_exists='replace', progress_bar=True)

100%|██████████| 1/1 [00:00<00:00, 2676.65it/s]
